# Generation Module Verification

This notebook tests the `src.generation` module which provides:
- **Query Rewriter** - Converts context-dependent questions to standalone form
- **Generator** - Produces answers with I_DONT_KNOW fallback
- **Retrieval Grader** - CRAG component for document relevance
- **Hallucination Grader** - Self-RAG component for answer grounding

**Note:** Full tests require GPU (Llama 3.1 8B). Run on Kaggle for complete testing.

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))

from src.generation import (
    create_generation_components,
    GenerationComponents,
    GradeDocuments,
    GradeHallucinations
)

## Step 1: Verify Pydantic Models

These define the JSON output schemas for graders.

In [ ]:
print("Pydantic Models:")
print(f"   • GradeDocuments: {list(GradeDocuments.model_fields.keys())}")
print(f"   • GradeHallucinations: {list(GradeHallucinations.model_fields.keys())}")

# Test instantiation
doc_grade = GradeDocuments(binary_score="yes")
hal_grade = GradeHallucinations(binary_score="no")

print(f"\n   Example GradeDocuments: {doc_grade.model_dump()}")
print(f"   Example GradeHallucinations: {hal_grade.model_dump()}")

print("\nPydantic models work!")

## Step 2: Verify GenerationComponents Dataclass

In [ ]:
import dataclasses

print("GenerationComponents:")
if dataclasses.is_dataclass(GenerationComponents):
    fields = [f.name for f in dataclasses.fields(GenerationComponents)]
    print(f"   • Type: dataclass")
    print(f"   • Fields: {fields}")
else:
    print(f"   • Type: {type(GenerationComponents).__name__}")

print("\nGenerationComponents structure verified!")

## Step 3: Test Factory Function (GPU Required)

Load Llama model and create components.

In [ ]:
# Use smaller model for faster testing
MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct"

print(f"Loading {MODEL_ID}...")
components = create_generation_components(model_id=MODEL_ID)

print(f"\nComponents Created:")
print(f"   • llm: {type(components.llm).__name__}")
print(f"   • query_rewriter: {type(components.query_rewriter).__name__}")
print(f"   • generator: {type(components.generator).__name__}")
print(f"   • retrieval_grader: {type(components.retrieval_grader).__name__}")
print(f"   • hallucination_grader: {type(components.hallucination_grader).__name__}")

print("\nAll components initialized!")

## Step 4: Test Chains

In [ ]:
# Test Query Rewriter
print("Testing Query Rewriter...")

# NOTE: query_rewriter expects 'messages' key (not 'chat_history')
result = components.query_rewriter.invoke({
    "messages": [("user", "Who is the CEO of Apple?"), ("assistant", "Tim Cook.")],
    "question": "How old is he?"
})

print(f"   Input: 'How old is he?' (with context about Tim Cook)")
print(f"   Output: '{result}'")
print("Query Rewriter works!")

In [ ]:
# Test Generator
print("Testing Generator...")

result = components.generator.invoke({
    "context": "Tim Cook is the CEO of Apple. He was born on November 1, 1960.",
    "question": "Who is Tim Cook?"
})

print(f"   Question: 'Who is Tim Cook?'")
print(f"   Answer: '{result[:200]}'")
print("Generator works!")

In [ ]:
# Test Retrieval Grader
print("Testing Retrieval Grader...")

result = components.retrieval_grader.invoke({
    "document": "Tim Cook is the CEO of Apple since 2011.",
    "question": "Who is the CEO of Apple?"
})

print(f"   Document relevant? {result}")
print("Retrieval Grader works!")

# Test Hallucination Grader
print("\nTesting Hallucination Grader...")

result = components.hallucination_grader.invoke({
    "documents": "Tim Cook was born on November 1, 1960.",
    "generation": "Tim Cook is currently 64 years old."
})

print(f"   Answer supported? {result}")
print("Hallucination Grader works!")

## Summary

**Components verified:**
- Pydantic models (GradeDocuments, GradeHallucinations)
- GenerationComponents dataclass
- Query Rewriter chain
- Generator chain
- Retrieval Grader chain
- Hallucination Grader chain